In [2]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import time

In [3]:
# Code Generation
nr_codewords = int(1e6)
bits_info = torch.randint(2, (nr_codewords, 4), dtype=torch.int)

print(bits_info)

tensor([[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 1, 1, 0],
        [0, 1, 0, 1],
        [0, 1, 0, 1],
        [0, 1, 1, 0],
        [0, 0, 1, 0],
        [1, 1, 1, 1],
        [1, 0, 0, 1],
        [1, 1, 1, 1],
        [1, 1, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 1, 0],
        [1, 1, 1, 1],
        [1, 1, 0, 1],
        [0, 0, 0, 0],
        [1, 0, 1, 0],
        [0, 1, 0, 1],
        [1, 0, 0, 1],
        [0, 0, 1, 0],
        [1, 1, 1, 1],
        [1, 0, 0, 1],
        [1, 0, 0, 1],
        [0, 1, 1, 0],
        [1, 1, 0, 1],
        [1, 1, 1, 1],
        [1, 1, 0, 1],
        [0, 0, 1, 0],
        [0, 1, 1, 1],
        [1, 1, 1, 0],
        [1, 0, 0, 1],
        [1, 1, 1, 1],
        [1, 0, 0, 1],
        [1, 0, 1, 1],
        [1, 0, 1, 1],
        [1, 1, 1, 0],
        [0, 1, 0, 0],
        [0, 1, 1, 0],
        [1, 0, 0, 1],
        [0, 0, 0, 0],
        [0, 0, 1, 1],
        [1, 0, 0, 1],
        [0, 1, 1, 1],
        [0, 0, 0, 1],
        [1, 1, 1, 0],
        [1

Hamming(7,4) Encoder

In [4]:
class hamming_encode(torch.nn.Module):
    def __init__(self):
        """
            Use Hamming(7,4) to encode the data.
    
        Args:
            data: data received from the Hamming(7,4) encoder(Tensor)
            generator matrix: generate the parity code
    
        Returns:
            encoded data: 4 bits original info with 3 parity code.
        """
        super(hamming_encode, self).__init__()

        # Define the generator matrix for Hamming(7,4)
        self.generator_matrix = torch.tensor([
            [1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1],
            [1, 1, 0, 1],
            [1, 0, 1, 1],
            [0, 1, 1, 1],
        ], dtype=torch.int)

    def forward(self, input_data):
        # Ensure input_data has shape (batch_size, 4)
        assert input_data.size(1) == self.generator_matrix.shape[1], "Input data must have same generator matrix row number bits."

        # Perform matrix multiplication to encode the data
        encoded_data = torch.matmul(input_data, self.generator_matrix.t()) % 2

        return encoded_data

In [5]:
encoder = hamming_encode()
encoded_codeword = encoder(bits_info)
print(encoded_codeword)

tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 0, 1, 1, 0],
        [0, 1, 0, 1, 0, 1, 0],
        [0, 1, 0, 1, 0, 1, 0],
        [0, 1, 1, 0, 1, 1, 0],
        [0, 0, 1, 0, 0, 1, 1],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 0, 0, 1, 0, 0, 1],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 1, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 1, 1],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 1, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 0, 1],
        [0, 1, 0, 1, 0, 1, 0],
        [1, 0, 0, 1, 0, 0, 1],
        [0, 0, 1, 0, 0, 1, 1],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 0, 0, 1, 0, 0, 1],
        [1, 0, 0, 1, 0, 0, 1],
        [0, 1, 1, 0, 1, 1, 0],
        [1, 1, 0, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 1, 0, 1, 1, 0, 0],
        [0, 0, 1, 0, 0, 1, 1],
        [0, 1, 1, 1, 0, 0, 1],
        [1, 1, 1, 0, 0, 0, 0],
        [1, 0, 0, 1, 0, 0, 1],
        [1, 1, 1, 1, 1, 1, 1],
        

BPSK Modulator + Noise

In [6]:
class bpsk_modulator(torch.nn.Module):
    def __init__(self):
        """
        Use BPSK to compress the data, which is easily to transmit.

        Args:
            codeword: data received from the Hamming(7,4) encoder(Tensor)
    
        Returns:
            data: Tensor contain all data modulated and add noise
        """
        super(bpsk_modulator, self).__init__()
        
    def forward(self, codeword, snr_dB):
        
        # data = torch.tensor(data, dtype=float)
        data = codeword.to(dtype=torch.float)
    
        for i in range(data.shape[0]):
            bits = data[i]
            bits = 2 * bits - 1
        
            # Add Gaussian noise to the signal
            noise_power = torch.tensor(10**(snr_dB / 10))
            noise = torch.sqrt(1/(2*noise_power)) * torch.randn(len(bits))
            noised_signal = bits + noise
            # noised_signal = bits
            data[i] = noised_signal
    
       
        return data

In [7]:
snr_dB = 15  # Signal-to-noise ratio in dB

# Modulate the signal
modulator = bpsk_modulator()
modulated_noise_signal = modulator(encoded_codeword, snr_dB)
print(modulated_noise_signal)

tensor([[-1.1352, -1.0980, -0.9837, -1.1463, -1.1204, -1.0384, -0.9366],
        [-0.9073, -1.0549, -0.9318, -1.0025, -1.0724, -0.9923, -0.9366],
        [-1.0144,  0.9858,  0.9372, -0.8775,  0.8964,  0.9976, -1.1846],
        [-1.0225,  1.0197, -1.0284,  0.9770, -0.9418,  1.0283, -1.2422],
        [-0.9077,  0.7060, -1.0224,  0.9977, -0.9837,  0.9235, -1.2112],
        [-1.1117,  0.8725,  1.0812, -0.6272,  1.0399,  1.0840, -1.0925],
        [-1.2038, -1.0892,  1.2236, -0.7798, -0.8581,  0.9433,  1.1861],
        [ 1.0055,  0.9067,  0.9152,  1.3410,  1.0982,  0.9769,  0.9506],
        [ 1.0933, -0.9262, -0.6796,  0.9709, -1.1544, -0.9640,  1.0295],
        [ 0.9716,  0.8656,  1.3104,  0.8885,  0.9387,  0.9713,  0.8218],
        [ 0.9150,  1.0016, -1.2748, -0.9610, -1.1087,  0.9474,  1.0594],
        [-1.0594, -1.0245, -1.2552, -1.0842, -0.8035, -0.9322, -1.1454],
        [-1.0107, -0.9505,  0.9399, -1.2346, -1.0923,  0.9486,  0.7814],
        [ 0.7776,  1.1090,  1.1899,  0.9461,  1.072

LLR Log-likelihood
y = s + n
Assuming that \( s \) is equally likely to be 0 or 1, and \( n \) is Gaussian with zero mean and variance \( N_0/2 \), where \( N_0 \) is the noise power spectral density.


In [8]:
def llr(signal, snr):
    """
    Calculate Log Likelihood Ratio (LLR) for a simple binary symmetric channel.

    Args:
        signal (torch.Tensor): Received signal from BPSK.
        noise_std (float): Standard deviation of the noise.

    Returns:
        llr: Log Likelihood Ratio (LLR) values.
    """
    
    # Assuming Binary Phase Shift Keying (BPSK) modulation
    noise_std = torch.sqrt(torch.tensor(10**(snr / 10)))

    # Calculate the LLR
    llr = 2 * signal * noise_std

    # return llr_values, llr
    return llr


In [9]:
llr_output = llr(modulated_noise_signal, snr_dB)
print("LLR values:", llr_output)

LLR values: tensor([[-12.7673, -12.3493, -11.0629, -12.8922, -12.6010, -11.6792, -10.5337],
        [-10.2046, -11.8638, -10.4794, -11.2746, -12.0616, -11.1603, -10.5336],
        [-11.4087,  11.0867,  10.5407,  -9.8686,  10.0820,  11.2199, -13.3235],
        [-11.5003,  11.4688, -11.5664,  10.9886, -10.5921,  11.5646, -13.9704],
        [-10.2083,   7.9403, -11.4991,  11.2205, -11.0636,  10.3865, -13.6222],
        [-12.5032,   9.8131,  12.1599,  -7.0539,  11.6951,  12.1918, -12.2876],
        [-13.5393, -12.2500,  13.7620,  -8.7703,  -9.6510,  10.6093,  13.3394],
        [ 11.3085,  10.1975,  10.2930,  15.0822,  12.3513,  10.9873,  10.6915],
        [ 12.2964, -10.4163,  -7.6439,  10.9191, -12.9828, -10.8422,  11.5785],
        [ 10.9279,   9.7353,  14.7378,   9.9925,  10.5575,  10.9240,   9.2430],
        [ 10.2904,  11.2646, -14.3377, -10.8085, -12.4688,  10.6557,  11.9145],
        [-11.9143, -11.5222, -14.1174, -12.1935,  -9.0363, -10.4838, -12.8821],
        [-11.3670, -10.6903,

LDPC Decoder

Strange behavior recording:
In 100 7-bit codewords, the speed on MPS(GPU) is slower than CPU. Reason unknown.

In [13]:
class LDPCBeliefPropagation(torch.nn.Module):
    def __init__(self, H):
        """
        LDPC Belief Propagation.
    
        Args:
            H: Low density parity code for building tanner graph.
            llr: Log Likelihood Ratio (LLR) values. Only for 7-bit codeword.
    
        Returns:
            estimated_bits: the output result from belief propagation.
        """
        
        super(LDPCBeliefPropagation, self).__init__()
        self.H = H
        self.num_check_nodes, self.num_variable_nodes = H.shape

        # Initialize messages
        self.messages_v_to_c = torch.ones((self.num_variable_nodes, self.num_check_nodes),dtype=torch.float)
        self.messages_c_to_v = torch.zeros((self.num_check_nodes, self.num_variable_nodes),dtype=torch.float)

    def forward(self, llr, max_iter):
        for iteration in range(max_iter):
            # Variable to check node messages
            for i in range(self.num_variable_nodes):
                for j in range(self.num_check_nodes):
                    # Compute messages from variable to check nodes
                    connected_checks = self.H[j, :] == 1
                    product = torch.prod(torch.tanh(0.5 * self.messages_v_to_c[connected_checks, j]))
                    self.messages_v_to_c[i, j] = torch.sign(llr[i]) * product

            # Check to variable node messages
            for i in range(self.num_check_nodes):
                for j in range(self.num_variable_nodes):
                    # Compute messages from check to variable nodes
                    connected_vars = self.H[:, j] == 1
                    sum_msgs = torch.sum(self.messages_c_to_v[connected_vars, i]) - self.messages_v_to_c[j, i]
                    self.messages_c_to_v[i, j] = 2 * torch.atan(torch.exp(0.5 * sum_msgs))

        # Calculate the final estimated bits and only take first four bits
        estimated_bits = torch.sign(llr) * torch.prod(torch.tanh(0.5 * self.messages_c_to_v), dim=0)
        estimated_bits = torch.where(estimated_bits>0, torch.tensor(1), torch.tensor(0))
        estimated_bits = estimated_bits[0:4]
        estimated_bits = estimated_bits.to(dtype=torch.int)


        return estimated_bits
    

In [15]:
# Define LDPC parameters
H = torch.tensor([ [1, 1, 1, 0, 0, 0, 0],
                   [0, 0, 1, 1, 1, 0, 0],
                   [0, 1, 0, 0, 1, 1, 0],
                   [1, 0, 0, 1, 0, 0, 1],])
iter = 10
ldpc_bp = LDPCBeliefPropagation(H)
n = 2000 # For n iteration, print time

# Store the final result from LDPC
tensor_size = torch.Size([llr_output.shape[0], 4])
final_result = torch.zeros(tensor_size)

#loop all the llr and get result.
for i in range(llr_output.shape[0]):
    start_time = time.time()
    
    bp_data = llr_output[i]
    estimated_bits = ldpc_bp(bp_data, iter)
    final_result[i] = estimated_bits
    
    end_time = time.time()
    iteration_time = (end_time - start_time)*n
    
    if i>0 and i % n == 0:
        print(f"Procedure {i}, Time taken for {n} 7-bit codeword: {iteration_time} seconds")

print(final_result)

tensor([[-0.6836, -0.6836, -0.6836, -0.6836, -0.6836, -0.5875, -0.5875],
        [-0.6836, -0.6836, -0.6836, -0.6836, -0.6836, -0.5875, -0.5875],
        [-0.6836,  0.6836,  0.6836, -0.6836,  0.6836,  0.5875, -0.5875],
        [-0.6836,  0.6836, -0.6836,  0.6836, -0.6836,  0.5875, -0.5875],
        [-0.6836,  0.6836, -0.6836,  0.6836, -0.6836,  0.5875, -0.5875],
        [-0.6836,  0.6836,  0.6836, -0.6836,  0.6836,  0.5875, -0.5875],
        [-0.6836, -0.6836,  0.6836, -0.6836, -0.6836,  0.5875,  0.5875],
        [ 0.6836,  0.6836,  0.6836,  0.6836,  0.6836,  0.5875,  0.5875],
        [ 0.6836, -0.6836, -0.6836,  0.6836, -0.6836, -0.5875,  0.5875],
        [ 0.6836,  0.6836,  0.6836,  0.6836,  0.6836,  0.5875,  0.5875],
        [ 0.6836,  0.6836, -0.6836, -0.6836, -0.6836,  0.5875,  0.5875],
        [-0.6836, -0.6836, -0.6836, -0.6836, -0.6836, -0.5875, -0.5875],
        [-0.6836, -0.6836,  0.6836, -0.6836, -0.6836,  0.5875,  0.5875],
        [ 0.6836,  0.6836,  0.6836,  0.6836,  0.683

Comparation and Plot

In [56]:
def calculate_ber(transmitted_bits, origin_bits):
    # Ensure that both tensors have the same shape
    assert transmitted_bits.shape == origin_bits.shape, "Shapes of transmitted and received bits must be the same."

    # Calculate the bit errors
    errors = (transmitted_bits != origin_bits).sum().item()
    print(errors)

    # Calculate the Bit Error Rate (BER)
    ber = errors / transmitted_bits.numel()

    return ber

In [62]:
# Describe the data:
# bits_info: original signal
decoded_bits = final_result #output from Maximum Likelihood
# decoded_bits = llr_output # Output from log-likelihood
# decoded_bits = 

ber = calculate_ber(decoded_bits, bits_info)
# print(ber)

print(final_result)


0
tensor([[1., 0., 0., 0.],
        [0., 1., 1., 0.],
        [1., 0., 1., 0.],
        ...,
        [1., 0., 1., 1.],
        [1., 1., 1., 1.],
        [0., 1., 1., 1.]])
